In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
import torchtext.experimental
import torchtext.experimental.vectors

import collections
import random
import time

In [2]:
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

In [4]:
print(raw_train_data)

In [5]:
raw_train_data = list(raw_train_data)

print(raw_train_data[0])

(0, 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between

In [6]:
raw_test_data = list(raw_test_data)

print(raw_test_data[0])

(0, 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they 

In [7]:
print(f'Number of training examples: {len(list(raw_train_data))}')
print(f'Number of testing examples: {len(list(raw_test_data))}')

Number of training examples: 25000
Number of testing examples: 25000


In [8]:
def get_train_valid_split(raw_train_data, split_ratio = 0.8):
       
    random.shuffle(raw_train_data)
        
    n_train_examples = int(len(raw_train_data) * split_ratio)
        
    valid_data = raw_train_data[:n_train_examples]
    train_data = raw_train_data[n_train_examples:]
    
    return train_data, valid_data

In [9]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [10]:
print(f'Number of training examples: {len(list(raw_train_data))}')
print(f'Number of validation examples: {len(list(raw_valid_data))}')
print(f'Number of testing examples: {len(list(raw_test_data))}')

Number of training examples: 5000
Number of validation examples: 20000
Number of testing examples: 25000


In [11]:
class Tokenizer:
    def __init__(self, tokenize_fn = 'basic_english', lower = True, max_length = None):
        
        self.tokenize_fn = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, s):
        
        tokens = self.tokenize_fn(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.max_length is not None:
            tokens = tokens[:max_length]
            
        return tokens

In [12]:
max_length = 250

tokenizer = Tokenizer(max_length = max_length)

In [13]:
s = "this film is terrible. i hate it and it's bad!"

print(tokenizer.tokenize(s))

['this', 'film', 'is', 'terrible', '.', 'i', 'hate', 'it', 'and', 'it', "'", 's', 'bad', '!']


In [14]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
    
    token_freqs = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freqs.update(tokens)
                
    vocab = torchtext.vocab.Vocab(token_freqs, **vocab_kwargs)
    
    return vocab

In [15]:
min_freq = 2
max_size = 25_000

vocab = build_vocab_from_data(raw_train_data, tokenizer, min_freq = min_freq, max_size = max_size)

In [16]:
vocab.freqs.most_common(20)

[('the', 48049),
 ('.', 46847),
 (',', 38799),
 ('a', 23646),
 ('and', 23193),
 ('of', 20752),
 ('to', 19099),
 ("'", 18536),
 ('is', 15569),
 ('it', 13913),
 ('i', 13864),
 ('in', 13151),
 ('this', 11868),
 ('that', 10177),
 ('s', 8460),
 ('was', 7508),
 ('movie', 6964),
 ('as', 6337),
 ('with', 6184),
 ('for', 6092)]

In [17]:
vocab.itos[:10]

['<unk>', '<pad>', 'the', '.', ',', 'a', 'and', 'of', 'to', "'"]

In [18]:
vocab.stoi['the']

2

In [19]:
def process_raw_data(raw_data, tokenizer, vocab):
    
    data = []
    
    for label, text in raw_data:
        
        tokens = tokenizer.tokenize(text)
        indexes = [vocab.stoi[token] for token in tokens]
        label = torch.LongTensor([label])
        indexes = torch.LongTensor(indexes)
        data.append((label, indexes))
        
    return data

In [20]:
train_data = process_raw_data(raw_train_data, tokenizer, vocab)

In [21]:
label, indexes = train_data[0]

print(indexes)

tensor([   12,    71,     7,   389,    14,    23,     5,  1526,    53,     7,
          674,     4,    22,    12,    95,   226,    11,     5,   120,     3,
          269,    34,    10,    15,  8559,  5795,  4922,   823,    13,     2,
           23,     4,    70,    10,    29,    65,     5,   269,     3,   231,
          240,    10,    15, 11384,   823,    13,    14,    23,     6,   138,
            5,     0,    20,  8257,     3,    12,   119, 11384,    59,    86,
            6,    31,     5,  1233,    15,    62,    10,     5,    83,   547,
            3,    70,    10,   137,    12,   166,     9,    27,   322,    50,
           13,    14,    18,  3717,  8257,     4,    22,    98,    67,   199,
          825,     3,   606,   220,     7,     2,  4794,   151,    15,    99,
           14,    23,  8394,     4,  1458,   160,   653,    47,    14,    23,
            3,    10,    11,    55,    49,    79,     4,    12,   110,    98,
            5,   358,  1512,    18,    15,    99,    76,  5469, 

In [22]:
print([vocab.itos[i] for i in indexes])

['i', 'would', 'of', 'given', 'this', 'film', 'a', 'zero', 'out', 'of', 'ten', ',', 'but', 'i', 'will', 'give', 'it', 'a', 'two', '.', 'reason', 'one', 'is', 'that', 'shah', 'rukh', 'khan', 'appears', 'in', 'the', 'film', ',', 'which', 'is', 'not', 'really', 'a', 'reason', '.', 'last', 'point', 'is', 'that', 'rani', 'appears', 'in', 'this', 'film', 'and', 'does', 'a', '<unk>', 'with', 'kamal', '.', 'i', 'love', 'rani', 'very', 'much', 'and', 'have', 'a', 'respect', 'that', 'she', 'is', 'a', 'great', 'actress', '.', 'which', 'is', 'why', 'i', 'didn', "'", 't', 'enjoy', 'her', 'in', 'this', 'movie', 'kissing', 'kamal', ',', 'but', 'its', 'no', 'big', 'deal', '.', 'anyway', 'enough', 'of', 'the', 'bedroom', 'scenes', 'that', 'made', 'this', 'film', 'noticeable', ',', 'lets', 'actually', 'talk', 'about', 'this', 'film', '.', 'is', 'it', 'good', 'or', 'bad', ',', 'i', 'think', 'its', 'a', 'completely', 'rubbish', 'movie', 'that', 'made', 'me', 'yawn', '.', 'me', 'being', 'a', 'fantastic', '

In [23]:
valid_data = process_raw_data(raw_valid_data, tokenizer, vocab)
test_data = process_raw_data(raw_test_data, tokenizer, vocab)

In [24]:
class Collator:
    def __init__(self, pad_idx):
        
        self.pad_idx = pad_idx
        
    def collate(self, batch):
        
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)
        
        text = nn.utils.rnn.pad_sequence(text, padding_value = self.pad_idx)
        
        return labels, text

In [25]:
pad_idx = vocab['<pad>']
collator = Collator(pad_idx)

In [26]:
batch_size = 128


train_iterator = torch.utils.data.DataLoader(train_data, 
                                             batch_size, 
                                             shuffle = True, 
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size, 
                                             shuffle = False, 
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size, 
                                            shuffle = False, 
                                            collate_fn = collator.collate)

In [27]:
class NBOW(nn.Module):
    def __init__(self, input_dim, emb_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.fc = nn.Linear(emb_dim, output_dim)
        
    def forward(self, sequence):
        
        # sequence = [seq len, batch size]
        
        embedded = self.embedding(sequence)
        
        # embedded = [seq len, batch size, emb dim]
        
        pooled = embedded.mean(0)
        
        # pooled = [batch size, emb dim]
        
        output = self.fc(pooled)
        
        # output = [batch size, output dim]
        
        return output

In [28]:
input_dim = len(vocab)
emb_dim = 100
output_dim = 2

model = NBOW(input_dim, emb_dim, output_dim, pad_idx)

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [30]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,007,202 trainable parameters


In [31]:
glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)

In [32]:
glove.vectors['the']

tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  0.8278,  0.27

In [33]:
glove.vectors['shoggoth']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [34]:
glove.vectors['The']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [35]:
def get_pretrained_embedding(vectors, vocab, unk_token):
    
    unk_vector = vectors[unk_token]
    
    emb_dim = unk_vector.shape[-1]
    
    pretrained_embedding = torch.zeros(len(vocab), emb_dim) 
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        pretrained_vector = vectors[token]
        pretrained_embedding[idx] = pretrained_vector
        if torch.all(torch.eq(pretrained_vector, unk_vector)):
            unk_tokens.append(token)
        
    return pretrained_embedding, unk_tokens

In [36]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(glove.vectors, vocab, unk_token)

In [37]:
pretrained_embedding

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.6815,  0.5909, -0.7525,  ...,  0.3713, -0.2969,  0.0298],
        [-0.5175, -0.5990,  0.3284,  ..., -0.9506, -0.4309,  0.5237],
        [ 0.0715, -0.1643, -0.1323,  ..., -0.0919, -0.6174, -0.0966]])

In [38]:
len(unk_tokens)

676

In [39]:
print(unk_tokens[:10])

['<unk>', '<pad>', '\x96', 'hadn', '****', '*1/2', 'camera-work', '-the', '100%', '***spoilers***']


In [40]:
model.embedding.weight.data

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [-1.6673,  0.3275, -0.1615,  ...,  1.3987,  1.1857,  0.3176],
        [ 0.8234, -2.1307, -1.2560,  ..., -3.2311, -1.3947,  0.7691],
        [ 1.5819, -0.5196, -0.5235,  ..., -1.6384,  0.4455,  0.8477]])

In [41]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.6815,  0.5909, -0.7525,  ...,  0.3713, -0.2969,  0.0298],
        [-0.5175, -0.5990,  0.3284,  ..., -0.9506, -0.4309,  0.5237],
        [ 0.0715, -0.1643, -0.1323,  ..., -0.0919, -0.6174, -0.0966]])

In [42]:
optimizer = optim.Adam(model.parameters())

In [43]:
criterion = nn.CrossEntropyLoss()

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using: {device}')

Using: cuda


In [45]:
model = model.to(device)
criterion = criterion.to(device)

In [46]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [47]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text in iterator:
        
        labels = labels.to(device)
        text = text.to(device)
        
        optimizer.zero_grad()
        
        predictions = model(text)
        
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [48]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [49]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [50]:
n_epochs = 25

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'nbow-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.690 | Train Acc: 53.61%
	 Val. Loss: 0.684 |  Val. Acc: 58.84%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.677 | Train Acc: 63.30%
	 Val. Loss: 0.672 |  Val. Acc: 64.32%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.660 | Train Acc: 68.20%
	 Val. Loss: 0.658 |  Val. Acc: 67.35%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.639 | Train Acc: 71.68%
	 Val. Loss: 0.639 |  Val. Acc: 68.81%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.613 | Train Acc: 73.05%
	 Val. Loss: 0.617 |  Val. Acc: 70.97%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.585 | Train Acc: 75.92%
	 Val. Loss: 0.594 |  Val. Acc: 72.90%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.552 | Train Acc: 78.71%
	 Val. Loss: 0.571 |  Val. Acc: 74.09%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.517 | Train Acc: 80.68%
	 Val. Loss: 0.546 |  Val. Acc: 76.50%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.483 | Train Acc: 82.44%
	 Val. Loss: 0.522 |  Val. Acc: 78.04%
Epoch: 10 | Epoch Time: 0m 0

In [51]:
model.load_state_dict(torch.load('nbow-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.380 | Test Acc: 83.80%


In [52]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [53]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

7.371189667537692e-07

In [54]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9999914169311523

In [55]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9331671595573425

In [56]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9331671595573425